In [ ]:
!pip install -U sentence_transformers

In [40]:
from sentence_transformers import SentenceTransformer, util

class ToolFinder:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)
        self.data = self.load_data()

    def load_data(self):
        # Load data here
        data = {
          "create_actionable_tasks_from_text": {
            "Description": "Given a text, extracts actionable insights, and creates tasks for them, which are kind of a work item.",
            "ArgumentName": {
              "text": {
                "Description": "The text from which the actionable insights need to be created.",
                "ArgumentType": "string"
              }
            }
          },
          "get_similar_work_items": {
            "Description": "Returns a list of work items that are similar to the given work item",
            "ArgumentName": {
              "work_id": {
                "Description": "The ID of the work item for which you want to find similar items",
                "ArgumentType": "string"
              }
            }
          },
          "get_sprint_id": {
            "Description": "Returns the ID of the current sprint",
            "ArgumentName": {
              "_": {
                "Description":"_",
                "ArgumentType":"_"
              }
            }
          },
          "prioritize_objects": {
            "Description": "Returns a list of objects sorted by priority. The logic of what constitutes priority for a given object is an internal implementation detail.",
            "ArgumentName": {
              "objects": {
                "Description": "A list of objects to be prioritized",
                "ArgumentType": "array of objects"
              }
            }
          },
          "search_object_by_name": {
            "Description": "Given a search string, returns the id of a matching object in the system of record. If multiple matches are found, it returns the one where the confidence is highest.",
            "ArgumentName": {
              "query": {
                "Description": "The search string, could be for example customer\u2019s name, part name, user name.",
                "ArgumentType": "string"
              }
            }
          },
          "summarize_objects": {
            "Description": "Summarizes a list of objects. The logic of how to summarize a particular object type is an internal\nimplementation\ndetail.",
            "ArgumentName": {
              "objects": {
                "Description": "List of objects to summarize",
                "ArgumentType": "array of objects"
              }
            }
          },
          "who_am_i": {
            "Description": "Returns the ID of the current user",
            "ArgumentName": {
              "_": {
                "Description":"_",
                "ArgumentType":"_"
              }
            }
          },
          "works_list": {
            "Description": "Returns a list of work items matching the request",
            "ArgumentName": {
              "applies_to_part": {
                "Description": "Filters for work belonging to any of the provided parts",
                "ArgumentType": "array of strings"
              },
              "created_by": {
                "Description": "Filters for work created by any of these users",
                "ArgumentType": "array of strings"
              },
              "issue.priority": {
                "Description": "Filters for issues with any of the provided priorities. Allowed values: p0, p1, p2,  p3",
                "ArgumentType": "array of strings"
              },
              "issue.rev_orgs": {
                "Description": "Filters for issues with any of the provided Rev organizations",
                "ArgumentType": "array of strings"
              },
              "limit": {
                "Description": "The maximum number of works to return. The default is '50'",
                "ArgumentType": "integer (int32)"
              },
              "owned_by": {
                "Description": "Filters for work owned by any of these users",
                "ArgumentType": "array of strings"
              },
              "stage.name": {
                "Description": "Filters for records in the provided stage(s) by name",
                "ArgumentType": "array of strings"
              },
              "ticket.needs_response": {
                "Description": "Filters for tickets that need a response",
                "ArgumentType": "boolean"
              },
              "ticket.rev_org": {
                "Description": "Filters for tickets associated with any of the provided Rev organizations",
                "ArgumentType": "array of strings"
              },
              "ticket.severity": {
                "Description": "Filters for tickets with any of the provided severities. Allowed",
                "ArgumentType": "array of strings"
              },
              "_": {
                "Description": "values: blocker, high, low, medium",
                "ArgumentType":"_"
              },
              "ticket.source_channel": {
                "Description": "Filters for tickets with any of the provided source channels",
                "ArgumentType": "array of strings"
              },
              "type": {
                "Description": "Filters for work of the provided types. Allowed values: issue, ticket, task",
                "ArgumentType": "array of strings"
              }
            }
          },
          "add_work_items_to_sprint": {
            "Description": "Adds the given work items to the sprint",
            "ArgumentName": {
              "work_ids": {
                "Description": "A list of work item IDs to be added to the sprint.",
                "ArgumentType": "array of strings"
              },
              "sprint_id": {
                "Description": "The ID of the sprint to which the work items should be added",
                "ArgumentType": "str"
              }
            }
          }
        }
        return data

    def extract_arguments(self):
        # Extract argument names and descriptions from data
        arg_names = []
        arg_descriptions = []

        for key, value in self.data.items():
            tmp_arg_dsc = value.get("ArgumentName")
            for argName, argDesc in tmp_arg_dsc.items():
                if argDesc.get("Description") == "_":
                    arg_names.append(key)
                    arg_descriptions.append(value.get("Description"))
                else:
                    arg_names.append(argName)
                    arg_descriptions.append(argDesc.get("Description"))

        return arg_names, arg_descriptions

    def encode_data(self, arg_descriptions, query):
        # Encode argument descriptions and query using the provided model
        arg_encodings = self.model.encode(arg_descriptions)
        query_encoding = self.model.encode(query)

        return arg_encodings, query_encoding

    def find_optimal_answer(self, arg_encodings, query_encoding):
        # Find the optimal answer using cosine similarity
        cosine_similarity = util.cos_sim(arg_encodings, query_encoding)
        optimal_answer_index = int(cosine_similarity.argmax())

        return optimal_answer_index

    def find_results(self, optimal_answer_index, arg_descriptions):
        # Find the tool, argument name, and description based on the optimal answer
        answer_argument_description = arg_descriptions[optimal_answer_index]
        answer_argument_name = ""
        answer_tool = ""

        for key, value in self.data.items():
            tmp_arg_dsc = value.get("ArgumentName")
            for argName, argDesc in tmp_arg_dsc.items():
                if argDesc.get("Description") == answer_argument_description or (
                    argDesc.get("Description") == "_" and value.get("Description") == answer_argument_description
                ):
                    answer_tool = key
                    answer_argument_name = argName

        results = {
            "answer_tool": answer_tool,
            "answer_argument_name": answer_argument_name,
            "answer_argument_description": answer_argument_description,
        }

        return results

    def find_tool_from_query(self, query):
        # Main function to find the tool based on the query
        arg_names, arg_descriptions = self.extract_arguments()

        arg_encodings, query_encoding = self.encode_data(arg_descriptions, query)

        optimal_answer_index = self.find_optimal_answer(arg_encodings, query_encoding)

        results = self.find_results(optimal_answer_index, arg_descriptions)

        return results



In [41]:
tool_finder = ToolFinder()
query = "Priority List"
results = tool_finder.find_tool_from_query(query)
print(results)

{'answer_tool': 'prioritize_objects', 'answer_argument_name': 'objects', 'answer_argument_description': 'A list of objects to be prioritized'}
